# Paris Metropolitan, an evolution

## Database handling

In [4]:
#imports
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import folium
import random
import copy
import collections
from collections import Counter
from collections import defaultdict
from itertools import filterfalse
import time
import csv

In [5]:
#path to database
database_path = r'paris_metropolitan_an_evolution.geojson'

In [7]:
#load the database file
#geoJSON database
paris_metro_db = json.load(open(database_path))
#csv database
db_1908 = pd.read_csv("GeoJSON_files_maps/1908_stations.csv", sep=',')
db_1915 = pd.read_csv("GeoJSON_files_maps/1915_stations.csv", sep=',')
db_1950 = pd.read_csv("GeoJSON_files_maps/1950_stations.csv", sep=',')

In [18]:
#Group the three maps databases
paris_metro_db_csv = pd.concat([db_1908, db_1915], sort=False)
paris_metro_db_csv = pd.concat([paris_metro_db_csv, db_1950], sort=False)

In [28]:
paris_metro_db_csv = paris_metro_db_csv.drop(columns=['Open_Date'])

In [43]:
paris_metro_db_csv

,X,Y,id,Name,Line,Info,Open_date,Close_date,Start_map,End_map
0,2.283876,48.876717,19081,Porte Maillot,1,NaN,NaN,NaN,1908,1950
1,2.277594,48.872253,19082,Porte Dauphine,2 Nord,NaN,NaN,NaN,1908,1908
2,2.277466,48.870780,19083,Porte Dauphine,2 Nord,NaN,NaN,NaN,1908,1908
3,2.285599,48.869948,19084,Victor Hugo,2 Nord,NaN,NaN,NaN,1908,1908
4,2.288289,48.875327,19085,Obligado,1,NaN,NaN,NaN,1908,1915
5,2.295141,48.874751,19086,Etoile,2 Nord,NaN,NaN,NaN,1908,1908
6,2.298535,48.878081,19087,Ternes,2 Nord,NaN,NaN,NaN,1908,1908
7,2.303274,48.879170,19088,Courcelles,2 Nord,NaN,NaN,NaN,1908,1908
8,2.308141,48.880514,19089,Monceau,2 Nord,NaN,NaN,NaN,1908,1908
9,2.313073,48.881219,190810,Villiers,2 Nord,NaN,NaN,NaN,1908,1908


In [39]:
stations_list = pd.DataFrame(columns=['X','Y','id','Name','Line','Info','Open_date', 'Close_date', 'Start_map', 'End_map'])
for index, row in paris_metro_db_csv.iterrows():
    if row["Line"] == "2 Sud":
        station = row
        station['Line']='5'
        station['Start_map'] = 1915
        station['End_map'] = 1915
        stations_list = stations_list.append(row)
        station = row
        station['Line']='6'
        station['Start_map'] = 1950
        station['End_map'] = 1950
        stations_list = stations_list.append(row)
        
    if row["Line"] == "2 Nord":
        station = row
        station['Line']='2'
        station['Start_map'] = 1915
        station['End_map'] = 1950
        stations_list = stations_list.append(row)
        
paris_metro_db_csv = paris_metro_db_csv.append(stations_list)


In [42]:
file_name = "paris_metropolitan_evolution_db.csv"
paris_metro_db_csv.to_csv(file_name, sep='\t', encoding='utf-8')

In [4]:
#feature structure
station = {
    "type": "Feature",
    "properties" : {
        "id":"",
        "Name":"",
        "Line":"",
        "Info":"",
        "Start_map":"",
        "End_map":"",
        "Open_date":"",
        "Close_date":""
    },
    "geometry": {
        "type" : "Point",
        "coordinates" : []
    }
}

In [5]:
paris_metro_db

{'type': 'FeatureCollection',
 'name': 'paris_metropolitan_an_evolution',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4171'}},
 'features': [{'type': 'Feature',
   'properties': {'id': 19081,
    'Name': 'Porte Maillot',
    'Line': '1',
    'Info': None,
    'Open_date': None,
    'Close_date': None,
    'Start_map': 1908,
    'End_map': 1950},
   'geometry': {'type': 'Point',
    'coordinates': [2.283875687160487, 48.87671727241535]}},
  {'type': 'Feature',
   'properties': {'id': 19082,
    'Name': 'Porte Dauphine',
    'Line': '2 Nord',
    'Info': None,
    'Open_date': None,
    'Close_date': None,
    'Start_map': 1908,
    'End_map': 1908},
   'geometry': {'type': 'Point',
    'coordinates': [2.277593685385499, 48.87225307885228]}},
  {'type': 'Feature',
   'properties': {'id': 19083,
    'Name': 'Porte Dauphine',
    'Line': '2 Nord',
    'Info': None,
    'Open_date': None,
    'Close_date': None,
    'Start_map': 1908,
    'End_map': 1908},
   'geo

In [6]:
stations = []
i=0
station_found = 0

for feature in paris_metro_db["features"]:
    stations.append(feature)
    
for feature in paris_metro_db["features"]:
#Stations on line 2 Sud have to be duplicated and their line have to be changed to 5 (line name changed on 1915 map)
    if feature["properties"]["Line"] == "2 Sud":
        station["geometry"]["coordinates"] = feature["geometry"]["coordinates"]
        station["properties"]["id"] = feature["properties"]["id"]
        station["properties"]["Name"] = feature["properties"]["Name"]
        station["properties"]["Line"] = "5"
        station["properties"]["Info"] = feature["properties"]["Info"]
        station["properties"]["Start_map"] = 1915
        station["properties"]["End_map"] = 1915
        station["properties"]["Open_date"] = feature["properties"]["Open_date"]
        station["properties"]["Close_date"] = feature["properties"]["Close_date"]
        stations.append(copy.deepcopy(station))
#Stations on line 2 Sud have to be duplicated and their line have to be changed to 6 (line name changed on 1950 map)
        station["geometry"]["coordinates"] = feature["geometry"]["coordinates"]
        station["properties"]["id"] = feature["properties"]["id"]
        station["properties"]["Name"] = feature["properties"]["Name"]
        station["properties"]["Line"] = "6"
        station["properties"]["Info"] = feature["properties"]["Info"]
        station["properties"]["Start_map"] = 1950
        station["properties"]["End_map"] = 1950
        station["properties"]["Open_date"] = feature["properties"]["Open_date"]
        station["properties"]["Close_date"] = feature["properties"]["Close_date"]
        stations.append(copy.deepcopy(station))   
#Stations on line 2 Nord have to be duplicated and their line have to be changed to 2 (line name changed on 1915 map)        
    elif feature["properties"]["Line"] == "2 Nord":
        station["geometry"]["coordinates"] = feature["geometry"]["coordinates"]
        station["properties"]["id"] = feature["properties"]["id"]
        station["properties"]["Name"] = feature["properties"]["Name"]
        station["properties"]["Line"] = "2"
        station["properties"]["Info"] = feature["properties"]["Info"]
        station["properties"]["Start_map"] = 1915
        station["properties"]["End_map"] = 1950
        station["properties"]["Open_date"] = feature["properties"]["Open_date"]
        station["properties"]["Close_date"] = feature["properties"]["Close_date"]
        stations.append(copy.deepcopy(station))
    
paris_metro_db["features"] = stations
with open('paris_metropolitan_evolution_db.geojson', 'w') as f:
    json.dump(stations, f, indent="\t")
    print('done')

done
